# From normalize notebook

In [1]:
gbif_ecoregion_gdf = (
    eco_gdf
    # Match the CRS of the GBIF data and the ecoregions
    .to_crs(gbif_gdf.crs)
    # Find ecoregion for each observation
    .sjoin(
        gbif_gdf,
        how='inner', 
        predicate='contains')
)
gbif_ecoregion_gdf

NameError: name 'eco_gdf' is not defined

In [ ]:
occurrence_df = (
    gbif_ecoregion_gdf
    # Select only necessary columns
    # Reset index
    .reset_index()
    # For each ecoregion, for each month...
    .groupby(['ecoregion_id','month'])
    # ...count the name of occurrences
    .agg
    (occurrences=('gbifID', 'count'),
     area= ('area' ,'first'))
)
 
# Normalize by area
occurrence_df['density'] = (
    occurrence_df.occurrences
    / occurrence_df.area
)
# Get rid of rare observations (possible misidentification?)
occurrence_df = occurrence_df[occurrence_df.occurrences>1]

# Take the mean by ecoregion
mean_occurrences_by_ecoregion = (
    occurrence_df
    .groupby('ecoregion_id')
    .mean()
)
# Take the mean by month
mean_occurrences_by_month = (
    occurrence_df
    .groupby('month')
    .mean()
    
)

In [ ]:
# Normalize by space and time for sampling effort
# Was stuck on this becuase a lack of famliarity 
# Got past it and will remember for net time 
occurrence_df['norm_occurrences'] = (
    occurrence_df[['density']]
    /mean_occurrences_by_ecoregion[['density']]
    /mean_occurrences_by_month[['density']]
)
occurrence_df

In [ ]:
occurrence_df.reset_index().plot.scatter(
    x='month' , y='norm_occurrences' , c='ecoregion_id',
    logy=True
)

# From plot note book

In [ ]:
# Simplify the geometry to speed up processing
eco_gdf.geometry = eco_gdf.simplify(.01, preserve_topology=False)
# Change the CRS to Mercator for mapping
eco_gdf = eco_gdf.to_crs(ccrs.Mercator())

# Check that the plot runs in a reasonable amount of time
eco_gdf.hvplot(geo=True, crs=ccrs.Mercator())

In [ ]:
occurrence_df[['norm_occurrences']].head()

In [ ]:
# Join the occurrences with the plotting GeoDataFrame
occurrence_gdf = eco_gdf.join(occurrence_df[['norm_occurrences']])

# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = gbif_gdf.to_crs(ccrs.Mercator()).total_bounds

# Plot occurrence by ecoregion and month
migration_plot = (
    occurrence_gdf
    .to_crs(ccrs.Mercator())
    .hvplot(
        c='norm_occurrences',
        groupby='month',
        # Use background tiles
        geo=True, crs=ccrs.Mercator(), tiles='CartoLight',
        title="Veery Migration",
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=600,
        widget_location='bottom'
    )
)

#Save the plot
migration_plot.save('migration.html', embed=True)

# Show the plot
migration_plot